In [333]:
import os
from collections import OrderedDict

import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xr

% matplotlib inline

### WB Data Analysis

**<span style="color:red; background:yellow;"></span>**

**Data & Scope:** downloaded on (date/time)
Two types of datasets:

1) Population (easy)
* years 1960-2016
* definition: total # residents (Regardless of citizenship/legal status)
* sources: (mid-year value)

    1) United Nations Population Division. World Population Prospects
    
    2) Census reports and other statistical publications from national statistical offices
    
    3) Eurostat: Demographic Statistics
    
    4) United Nations Statistical Division. Population and Vita

2) GDP (slightly more tricky)
- years: 1960 - 2016
- choosing to look at `gdp` (market, constant) first.
- Other types that one can look at:
    * MKT current
    * PPP current
    * PPP constant

**Source:** World Bank

**Assumptions/Expectations**

**Analysis Goals**
- Task 3 Deliverable: ADM0 population & real income estimates from `1950-2017`  **<span style="color:gray; background:lime;">DONE</span>**

- Find out which countries are missing data for which years. 
    Population **<span style="color:gray; background:lime;">DONE</span>**
    GDP
- Or alternatively, which years are missing certain *countries*

Possible convenience functions:
* Find years missing for a given country (to check important countries)
* Create filter for population > 10 mil: look at countries exit/enter data on which years. (Mike's suggesiton)
* From metadata, get a list of country code that are not *countries* to filter them from the list **<span style="color:gray; background:lime;">DONE</span>**


**Conclusion**
1. Population. Population data exists from **1960 to 2016.**
    
```  
Countries out of 217 that are missing population data (nan):

West Bank and Gaza: 1960 - 1989
Serbia: 1960 - 1989
Sint Maarten (Dutch part): 1960 -1998
Kuwait: 1992-1994
Eritrea: 2012-2016

```

2. GDP 

```
Countries out of 217 that are missing gdp data

a. GPP market constant
b. GDO PPP constant

```


**Questions**
1. How are different sources of population data used to result in one final set? (are there any overlaps between 4 sources?) i.e. what is the methodology for compilation?

2. How often are population/income data updated?


**TOC**
* A. Methodology for printing missing years

* B. Convenience functions for filtering relevant columns and country data only

* C. Population Data Analysis

* D. Income Data Analysis

## A. Methodology for printing missing years for WB datasets

#### Do this:

0. Download dataframe and metadata

1. `filter_non_countries(df, metadata)`

2. `select_relevant_cols(df)`

3. `list_of_country_idx_with_nans = get_country_idx_with_nans(df)`

4.  `print_missing_years_for_all_countries(_df, _list_of_country_idx_with_nans)`

#### Detailed Explanation:

1) Filter irrelevant (aggregate, not country-specific) data from the dataframe using metadata


2) Filter irrelevant (aggregate, not country-specific) data from the dataframe using metadata


* Note - running 2/ after 1/ matters for keeping out extra data such as `IncomeGroup`, because 2/ filters out extra columns gained from merging with metadata in 1/ 

3) Get index (Row) list of countries that have any `nan` data points for existing years range (ex .1960...2016)  
* Assumes that `Country Name` and `Country Code` columns don't have missing data

4) Print out missing years for each country using the tamed dataframe and index list of countries with nans
* calls `print_missing_years_for_country` for each country



## B. Convenience functions for filtering relevant columns and country data only


Both population and income dataset includes list of countries (217) plus various other groups (46 - regional, income or political) as entry data. 217 is the expected number of countries listed in World Bank, 2017.

Convenience functions:

1) From the original WB dataset, select only columns of interest

2) Get only country-specific GDP and population data (filter out other groups^^, mentioned above, data)

### 1) Select interested columns from data

In [334]:
def select_relevant_cols(_df):
    # returns df with 'Country Name', 'Country Code' and all years (1960 to 2016)
    _cols = ['Country Name'] + ['Country Code'] + [str(yr) for yr in range(1960, 2017)]
    return _df[_cols]

### 2) Filter out non_countries (i.e. get only country data)

In [335]:
def filter_non_countries(_df, _metadata):
    '''
    _df : pd.DataFrame
        either income or population data
        
    _metadata : pd.DataFrame
        metadata on a list of entries including countries and non-countries 
        data source is from the World Bank
        has IncomeGroup column that is not null for countries (217)
    '''
    # Country Code in both dataframes
    _merged = _df.merge(_metadata, on='Country Code')
    # non-countries have no IncomeGroup
    non_country_mask = _merged['IncomeGroup'].isnull()
    merged_country_only = _merged[~non_country_mask]
    return merged_country_only

# C. Population data

Total population (absolute units) - based on national census and extrapolation and interpolation for missing values (based on data from United Nations, other census organizations, Eurostats and WB methodology). Subject to undercounting/biases for both high and low/mid income countries. 

Interpolation and extrapolation done by World Bank/UN (??-confirm the responsible party) for certain years/countries that are missing census data, or missing pre/post census information for given time frame. Uses demographic models, etc. 

### Initial survey of data

#### Download population data

In [336]:
# Have to run jupyter from WB directory
df2 = pd.read_csv('population/API_SP.POP.TOTL_DS2_en_csv_v2.csv', skiprows=4)

#del df2['Unnamed: 62'] # remove extraneous data
df2.columns
df2.tail(3)

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
261,South Africa,ZAF,"Population, total",SP.POP.TOTL,17396367.0,17850045.0,18322335.0,18809939.0,19308166.0,19813947.0,...,5.025581e+07,5.097943e+07,5.172935e+07,5.250652e+07,5.331196e+07,5.414673e+07,5.501198e+07,55908865.0,NaN,NaN
262,Zambia,ZMB,"Population, total",SP.POP.TOTL,3044846.0,3140264.0,3240587.0,3345145.0,3452942.0,3563407.0,...,1.345642e+07,1.385003e+07,1.426476e+07,1.469994e+07,1.515321e+07,1.562097e+07,1.610059e+07,16591390.0,NaN,NaN
263,Zimbabwe,ZWE,"Population, total",SP.POP.TOTL,3747369.0,3870756.0,3999419.0,4132756.0,4269863.0,4410212.0,...,1.381060e+07,1.408632e+07,1.438665e+07,1.471083e+07,1.505451e+07,1.541168e+07,1.577745e+07,16150362.0,NaN,NaN


#### Download metadata

**WARNING**: had to re-download the file because I modified it by
opening and interacting with in via Excel. Produced error in read_csv call.

**TIP for myself**: Only Open a copy version in Excel


In [337]:
meta_country = pd.read_csv('./population/Metadata_Country_API_SP.POP.TOTL_DS2_en_csv_v2.csv')

del meta_country['Unnamed: 5'] 
meta_country.columns

Index(['Country Code', 'Region', 'IncomeGroup', 'SpecialNotes', 'TableName'], dtype='object')

In [338]:
df2
df2['2017'].unique() # nan -> all 2017 values nan

array([ nan])

### Initial survey of metadata

In [339]:
meta_country['Region'].unique() # 7 excluding nan

'''['Latin America & Caribbean' 
 'South Asia' 
 'Sub-Saharan Africa'
 'Europe & Central Asia' /// nan 
 'Middle East & North Africa'
 'East Asia & Pacific' 
 'North America']

'''

#print(meta_country['IncomeGroup'].nunique()) #4
meta_country['IncomeGroup'].unique() 
'''['High income', 'Low income', 'Lower middle income', 'Upper middle income', nan]'''

meta_country['Country Code'].nunique() # 263 entries 
meta_country['IncomeGroup'].isnull().value_counts() # 217 False (countries) # 46 nan
# False values are countries -- i.e. all countries belong to an IncomeGroup
# Validation: we know from (World Bank 2017) that total of 217 countries were included

False    217
True      46
Name: IncomeGroup, dtype: int64

### Select relevant columns

In [340]:
df2 # includes Country Name and all years (1960-2016)
df2.shape # 264 entries, 58 columns 

(264, 63)

In [341]:
df2.head(5)   # Show top 5 rows

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,Unnamed: 62
0,Aruba,ABW,"Population, total",SP.POP.TOTL,54211.0,55438.0,56225.0,56695.0,57032.0,57360.0,...,101453.0,101669.0,102053.0,102577.0,103187.0,103795.0,104341.0,104822.0,NaN,NaN
1,Afghanistan,AFG,"Population, total",SP.POP.TOTL,8996351.0,9166764.0,9345868.0,9533954.0,9731361.0,9938414.0,...,28004331.0,28803167.0,29708599.0,30696958.0,31731688.0,32758020.0,33736494.0,34656032.0,NaN,NaN
2,Angola,AGO,"Population, total",SP.POP.TOTL,5643182.0,5753024.0,5866061.0,5980417.0,6093321.0,6203299.0,...,22549547.0,23369131.0,24218565.0,25096150.0,25998340.0,26920466.0,27859305.0,28813463.0,NaN,NaN
3,Albania,ALB,"Population, total",SP.POP.TOTL,1608800.0,1659800.0,1711319.0,1762621.0,1814135.0,1864791.0,...,2927519.0,2913021.0,2905195.0,2900401.0,2895092.0,2889104.0,2880703.0,2876101.0,NaN,NaN
4,Andorra,AND,"Population, total",SP.POP.TOTL,13411.0,14375.0,15370.0,16412.0,17469.0,18549.0,...,84462.0,84449.0,83751.0,82431.0,80788.0,79223.0,78014.0,77281.0,NaN,NaN


In [342]:
df2_countries_only = filter_non_countries(df2, meta_country)
df2_countries_only[200:205] # few U- countires

,Country Name,Country Code,Indicator Name,Indicator Code,1960,1961,1962,1963,1964,1965,...,2013,2014,2015,2016,2017,Unnamed: 62,Region,IncomeGroup,SpecialNotes,TableName
244,Uganda,UGA,"Population, total",SP.POP.TOTL,6788214.0,7006633.0,7240174.0,7487429.0,7746198.0,8014401.0,...,37553726.0,38833338.0,40144870.0,41487965.0,NaN,NaN,Sub-Saharan Africa,Low income,Fiscal year end: June 30; reporting period for...,Uganda
245,Ukraine,UKR,"Population, total",SP.POP.TOTL,42662149.0,43203635.0,43749470.0,44285899.0,44794327.0,45261935.0,...,45489600.0,45271947.0,45154029.0,45004645.0,NaN,NaN,Europe & Central Asia,Lower middle income,The new base year is 2010.,Ukraine
247,Uruguay,URY,"Population, total",SP.POP.TOTL,2538651.0,2571690.0,2603887.0,2635129.0,2665390.0,2694537.0,...,3408005.0,3419546.0,3431552.0,3444006.0,NaN,NaN,Latin America & Caribbean,High income,NaN,Uruguay
248,United States,USA,"Population, total",SP.POP.TOTL,180671000.0,183691000.0,186538000.0,189242000.0,191889000.0,194303000.0,...,316204908.0,318563456.0,320896618.0,323127513.0,NaN,NaN,North America,High income,Fiscal year end: September 30; reporting perio...,United States
249,Uzbekistan,UZB,"Population, total",SP.POP.TOTL,8549493.0,8837349.0,9138097.0,9454250.0,9788986.0,10143740.0,...,30243200.0,30757700.0,31298900.0,31848200.0,NaN,NaN,Europe & Central Asia,Lower middle income,NaN,Uzbekistan


In [343]:
df_pop = select_relevant_cols(df2_countries_only)
df_pop.shape

(217, 59)

### [F] Function: Get country indexes with nans

F for function

In [344]:
get_country_idx_with_nans = lambda _dd: pd.isnull(_dd).any(1).nonzero()[0]

### Find countries/years missing population data

In [345]:
# df_pop.isnull().sum()
# note that nan values per year changes from 1-4, 1989 and prior are all 4,
# after that there is a shifts six times:
# 4 (60-89) -> 2 (90-91)-> 3 (92-94) -> 2 (95-97) -> 1 (98-2011)-> 2 (2012-2016)-> 264 (2017)

# Compare two years (diff countries)


In [346]:
list_of_country_idx_with_nans = get_country_idx_with_nans(df_pop) # same as above
list_of_country_idx_with_nans

array([ 58, 106, 161, 176, 184])

NOTE on nans:

* `df.isnull().sum()`: returns total number of nan values for each column (whole df) <- what I used
* comparison of nan == np.nan to find nan values does not work (returns False)
* can use np.isnan(val) for a single value OR possibly use `apply` method for whole col (did not try)

In [347]:
def print_country_name_and_index(_df, _list_of_indexes_with_nans):
    _countries = [(_df.iloc[i]['Country Name'], i) for i in _list_of_indexes_with_nans]
    print (_countries)

print_country_name_and_index(df_pop, list_of_country_idx_with_nans)

[('Eritrea', 58), ('Kuwait', 106), ('West Bank and Gaza', 161), ('Serbia', 176), ('Sint Maarten (Dutch part)', 184)]


### Define `get_missing_years`

In [348]:
def print_missing_years_for_country(_df, idx):
    '''
    Calculate the number of years the country has no
    data points (ie. has nan value)
    
    Parameters
    ----------
    _df : pandas.DataFrame
        dataframe containing countries and these columns
        Country Name, Country Code, '1960'... '2016' 
        where year values can be either population, income, or NaN
        
    idx : Python list 
        list of of row indexes (countries) in the dataframe that has missing years
        
    Prints 
    -------
    For each country with missing years, prints country name, 
    followed by a Pandas.Index of missing years (each year is
    a String)
    
    ex. Eritrea Index(['2012', '2013', '2014', '2015', '2016'], dtype='object')
        Kuwait Index ...
        
    Note (additional complexity)
    ----
    Can return start/end missing years, and # missing years instead, 
    if the years are consecutive
    '''
    _country = _df.iloc[idx]
    _nan_years = _country.loc[_country.isnull()]
    print(_country['Country Name'], _nan_years.keys())\

In [349]:

def print_missing_years_for_all_countries(_df, _list_of_idx):
    for _i in _list_of_idx:
        print_missing_years_for_country(_df, _i)
        
print_missing_years_for_all_countries(df_pop, list_of_country_idx_with_nans)

def get_country_names_with_num_missing_years(_df, idx):
    '''
    Calculate the number of years the country has no
    data points (ie. has nan value)
    
    Parameters
    ----------
    _df : pandas.DataFrame
        dataframe containing countries and these columns
        Country Name, Country Code, '1960'... '2016' 
        where year values can be either population, income, or NaN
        
    idx : Python list 
        list of of country names (Str) in the dataframe that has missing years
    '''
    _country = _df.iloc[idx]
    _nan_years = _country.loc[_country.isnull()]
    print(_country['Country Name'], len(_nan_years))
    

Eritrea Index(['2012', '2013', '2014', '2015', '2016'], dtype='object')
Kuwait Index(['1992', '1993', '1994'], dtype='object')
West Bank and Gaza Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989'],
      dtype='object')
Serbia Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989'],
      dtype='object')
Sint Maarten (Dutch part) Index(['1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
  

### Manual validation

In [350]:
#df_pop.iloc[58] # Eritrea missing 2012-2016
#df_pop.iloc[106] # Kuwait missing 1992-1994
#df_pop.iloc[161] # West Bank and Gaza missing 1960 - 1989
#df_pop.iloc[176] # Serbia missing 1960 - 1989
# df_pop.iloc[184] # Sint Maarten (Dutch part) 1960 -1997

In [351]:
#df_pop['1990'].isnull().sum() == 1 # expect 1 (excluding West Bank/Serbia)
#df_pop['1993'].isnull().sum() == 2 # expect 2 (Kuwait missing too)
#df_pop['1995'].isnull().sum() == 1 # expect 1 (Kuwait back again)
#df_pop['1999'].isnull().sum() # expect 0 Sint Maarten back
#df_pop['2012'].isnull().sum() # expect 1 (Eritrea goes missing)
#df_pop['2016'].isnull().sum() # expect 1

In [352]:
# doing this manual painful way for now

((df_pop['1960'].isnull().sum() == 3) 
 and (df_pop['1990'].isnull().sum() == 1) 
 and (df_pop['1993'].isnull().sum() == 2) 
 and (df_pop['1995'].isnull().sum() == 1)
 and df_pop['1999'].isnull().sum()==0
 and df_pop['2012'].isnull().sum()==1
 and df_pop['2016'].isnull().sum()==1)

True

##  Playing around with data

#### Not used / unhelpful

In [353]:
# Get countries as dataframe (subset of original) by first letter
def get_by_first_letter(df, first_letter):
    return df.loc[df['Country Name'].str.startswith(first_letter)]

# Get country by name
def get_row_by_country_name(_df, country_name):
    return _df[_df['Country Name'] == country_name]

# Get country row from dataframe by index
def get_row_by_index(df, country_idx):
    return df.iloc[country_idx]

# Get all countries that start with S
s_countries = get_by_first_letter(df_pop, 'S')
#s_countries
    
# Get_by_first_letter(df, 'K') # example South Korea is index 124
# get_by_index(df, 124)


In [354]:
# Returns all values of the 'Country Name' column as an array
#df2['Country Name'].values #lists all countries
# United States
# China
# India

china = get_row_by_country_name(df2, 'China')
india = get_row_by_country_name(df2, 'India')
usa = get_row_by_country_name(df2, 'United States')
def get_first_and_last_years(_country):
    # _country is a dataframe (1 row)
    return _country[['1960', '2016']]
    
get_first_and_last_years(china)
get_first_and_last_years(usa)
get_first_and_last_years(india)

# manual confirmation: these values match online WB values (https://data.worldbank.org/indicator/SP.POP.TOTL?year_high_desc=true)

,1960,2016
107,449480608.0,1.324171e+09


## D. GDP Data Analysis

### 1/ Picked constant market prices GDP

#### 0. Download dataframe (data) and metadata

In [355]:
# Open files
# NY.GDP.PCAP.PP.KD is WDI indicator
'''
NY = national accounts: income
MKTP = market prices
PCAP = per capita
PP = purchasing power (no PP means not PP)
KD = constant (vs CD = current)
'''
# df_gdp is gdp market constant
gdp_mkt_const = pd.read_csv('./gdp/gdp_constant/API_NY.GDP.MKTP.KD_DS2_en_csv_v2.csv', skiprows=3)
gdp_meta = pd.read_csv('./gdp/gdp_constant/Metadata_Country_API_NY.GDP.MKTP.KD_DS2_en_csv_v2.csv')

In [356]:
# Get variables (column names)
gdp_mkt_const.columns # years 1960-2017
gdp_mkt_const.shape
del gdp_mkt_const['Unnamed: 62']

In [357]:
gdp_mkt_const.columns

Index(['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code',
       '1960', '1961', '1962', '1963', '1964', '1965', '1966', '1967', '1968',
       '1969', '1970', '1971', '1972', '1973', '1974', '1975', '1976', '1977',
       '1978', '1979', '1980', '1981', '1982', '1983', '1984', '1985', '1986',
       '1987', '1988', '1989', '1990', '1991', '1992', '1993', '1994', '1995',
       '1996', '1997', '1998', '1999', '2000', '2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017'],
      dtype='object')

#### 1. Filter non country/ aggregate groups data

Filtered df should have 217 rows and 68 columns

In [358]:
gdp_mkt_const_country_only = filter_non_countries(gdp_mkt_const, gdp_meta)
gdp_mkt_const_country_only.shape # Expect 217 rows (extra columns)

(217, 67)

In [359]:
gdp_mkt_const_country_only.columns.difference(gdp_mkt_const.columns)

Index(['IncomeGroup', 'Region', 'SpecialNotes', 'TableName', 'Unnamed: 5'], dtype='object')

#### 2. Select Relevant Columns from dataframe

Should return columns including Country Code, Country Name,  and all years (1960-2016)

Expect 217 rows and 59 columns

In [360]:
df_gdp = select_relevant_cols(gdp_mkt_const_country_only)
df_gdp.shape

(217, 59)

In [361]:
#df_gdp.columns # manual check

#### 3. Get list of country indexes with nanas

In [362]:
list_of_country_idx_with_nans_gdp = get_country_idx_with_nans(df_gdp)

In [363]:
len(list_of_country_idx_with_nans_gdp) # 131 countries!
#list_of_country_idx_with_nans_gdp

131

#### 4. Print missing years for all countries

In [364]:
# print_missing_years_for_all_countries(df_gdp, 
#                                      list_of_country_idx_with_nans_gdp
df_nans = df_gdp.loc[list_of_country_idx_with_nans_gdp]
df_nans.shape

(131, 59)

In [365]:
df_nans.columns

Index(['Country Name', 'Country Code', '1960', '1961', '1962', '1963', '1964',
       '1965', '1966', '1967', '1968', '1969', '1970', '1971', '1972', '1973',
       '1974', '1975', '1976', '1977', '1978', '1979', '1980', '1981', '1982',
       '1983', '1984', '1985', '1986', '1987', '1988', '1989', '1990', '1991',
       '1992', '1993', '1994', '1995', '1996', '1997', '1998', '1999', '2000',
       '2001', '2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009',
       '2010', '2011', '2012', '2013', '2014', '2015', '2016'],
      dtype='object')

### TASK: Get missing years/countries


### Ideas

1. Idea Red
    
    1) Get a list of countries that has any missing data, including # years missing
    
    2) Sort by years missing (country with largest missing data first)

2. Idea Orange (outcome data need)

    1) Check interesting countries
        Mexico
        Philipines
        USA
        Ethiopia
        Germany/West Germany
        Indonesia
        Soviet Union/Russia
        France
        Brazil
        Spain....
    
3. Idea Yellow (population as guage for important countries)
    1) Filter by population


### TASK: Get list of years missing for a country

Super naive EDA



In [366]:
# Count nan
print(sorted(df_gdp.isnull().sum(axis=1).unique().tolist()))

[0, 1, 2, 3, 5, 6, 7, 8, 10, 11, 13, 14, 15, 16, 17, 19, 20, 21, 22, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 40, 41, 42, 43, 44, 47, 49, 56, 57]


#### Number of missing years range from 0 to 57.. T.T

There are 17 unique values

### Approach 1: Create a filter for countries with > 10 mil pop (Mike's Suggestion)

Shelved for now, because all have higher than 10 mil pop??

In [367]:
df_gdp2 = df_gdp.copy()

# Reset Index 
df_gdp_reindexed = df_gdp2.reset_index(drop=True)

In [368]:
list_of_countries = [x for x in df_gdp_reindexed['Country Name'].values]
num_nan_per_country = lambda i: df_gdp_reindexed.loc[i].isnull().sum()

num_countries = len(list_of_countries) # 217

countries = {}    
countries = [{'name': list_of_countries[i], 
                  'num_missing': num_nan_per_country(i)} for i in range(0, num_countries)]
#countries
# form: [{'name': 'Aruba', 'num_missing': 56},

### Find and print gdp missing countries

In [369]:
import operator

countries_with_nans_and_idx = {}

for i in list_of_country_idx_with_nans_gdp:
    country_name = df_gdp.iloc[i]['Country Name']
    countries_with_nans_and_idx[country_name] = num_nan_per_country(i)

gdp_missing_countries = sorted(countries_with_nans_and_idx.items(), 
                               key=operator.itemgetter(1), 
                               reverse=True)

In [370]:
len(gdp_missing_countries) # 131 as expected

131

In [371]:
for a, b in gdp_missing_countries[0:14]:
    print(a)

Channel Islands
Curacao
Cayman Islands
Gibraltar
St. Martin (French part)
New Caledonia
Korea, Dem. People’s Rep.
French Polynesia
San Marino
Somalia
Sint Maarten (Dutch part)
Syrian Arab Republic
Turks and Caicos Islands
British Virgin Islands


In [372]:
gdp_missing_countries

[('Channel Islands', 57),
 ('Curacao', 57),
 ('Cayman Islands', 57),
 ('Gibraltar', 57),
 ('St. Martin (French part)', 57),
 ('New Caledonia', 57),
 ('Korea, Dem. People’s Rep.', 57),
 ('French Polynesia', 57),
 ('San Marino', 57),
 ('Somalia', 57),
 ('Sint Maarten (Dutch part)', 57),
 ('Syrian Arab Republic', 57),
 ('Turks and Caicos Islands', 57),
 ('British Virgin Islands', 57),
 ('Aruba', 56),
 ('Faroe Islands', 56),
 ('Liechtenstein', 56),
 ('Monaco', 56),
 ('South Sudan', 49),
 ('Nauru', 47),
 ('Libya', 44),
 ('American Samoa', 43),
 ('Guam', 43),
 ('Northern Mariana Islands', 43),
 ('Virgin Islands (U.S.)', 43),
 ('Afghanistan', 42),
 ('Maldives', 41),
 ('Qatar', 40),
 ('Sao Tome and Principe', 40),
 ('Timor-Leste', 40),
 ('Kosovo', 40),
 ('Eritrea', 37),
 ('Montenegro', 37),
 ('Haiti', 36),
 ('Estonia', 35),
 ('Croatia', 35),
 ('Lithuania', 35),
 ('Latvia', 35),
 ('Moldova', 35),
 ('Serbia', 35),
 ('Slovenia', 35),
 ('Bosnia and Herzegovina', 34),
 ('West Bank and Gaza', 34),
 

In [373]:
get_missing_num_years_for_country(df_gdp, 'Germany')

10

### get_missing_num_years_for_country(_df, _name)

Given dataframe adn country name, return how many missing data points the country has

In [384]:
# years only
_years = [str(yr) for yr in range(1960, 2017)]

get_row_by_country_name(df_gdp, 'Sri Lanka') # missing 1960

# Get missing years total for a given country
def get_missing_num_years_for_country(_df, _name):
    _yrs = [str(yr) for yr in range(1960, 2017)]
    return get_row_by_country_name(_df, _name)[_yrs].isnull().sum().value_counts()[1]

get_missing_num_years_for_country(df_gdp, 'Somalia')
get_missing_num_years_for_country(df_gdp, 'Germany')
get_missing_num_years_for_country(df_gdp, 'Ethiopia')
get_missing_num_years_for_country(df_gdp, 'Vietnam')
get_missing_num_years_for_country(df_gdp, 'Germany')


10

In [375]:
get_row_by_country_name(df_gdp, 'Germany')['1970'] # missing up to 1969
get_row_by_country_name(df_gdp, 'Russian Federation')['1988']

199   NaN
Name: 1988, dtype: float64

### get_max_countries(_df, _n, _yr):

given dataframe, number wanted and specific string, year, return largest values for that given year

In [376]:
def get_max_countries(_df, _n, _yr):
    return _df.nlargest(_n, _yr)

get_max_countries(df_gdp, 15, '2016')

,Country Name,Country Code,1960,1961,1962,1963,1964,1965,1966,1967,...,2007,2008,2009,2010,2011,2012,2013,2014,2015,2016
248,United States,USA,3.078071e+12,3.148867e+12,3.340948e+12,3.487949e+12,3.690250e+12,3.926426e+12,4.181644e+12,4.286185e+12,...,1.505540e+13,1.501149e+13,1.459484e+13,1.496437e+13,1.520402e+13,1.554216e+13,1.580286e+13,1.617746e+13,1.659745e+13,1.686560e+13
38,China,CHN,1.279381e+11,9.304941e+10,8.785725e+10,9.690655e+10,1.145242e+11,1.339360e+11,1.482002e+11,1.396490e+11,...,4.596580e+12,5.040347e+12,5.514130e+12,6.100620e+12,6.682403e+12,7.207390e+12,7.766513e+12,8.333287e+12,8.908301e+12,9.505157e+12
116,Japan,JPN,7.962132e+11,8.921054e+11,9.715829e+11,1.053911e+12,1.176973e+12,1.245470e+12,1.377970e+12,1.530679e+12,...,5.848026e+12,5.784079e+12,5.470749e+12,5.700098e+12,5.693517e+12,5.778636e+12,5.894237e+12,5.914022e+12,5.986140e+12,6.045913e+12
53,Germany,DEU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.441356e+12,3.478602e+12,3.283144e+12,3.417095e+12,3.542160e+12,3.559587e+12,3.577015e+12,3.634080e+12,3.696613e+12,3.765638e+12
75,France,FRA,6.081763e+11,6.416641e+11,6.844771e+11,7.210773e+11,7.680795e+11,8.047786e+11,8.467398e+11,8.864364e+11,...,2.669265e+12,2.674478e+12,2.595812e+12,2.646837e+12,2.701871e+12,2.706807e+12,2.722405e+12,2.748202e+12,2.777538e+12,2.810525e+12
79,United Kingdom,GBR,7.267386e+11,7.454417e+11,7.553411e+11,7.852243e+11,8.247861e+11,8.477892e+11,8.651630e+11,8.851566e+11,...,2.507592e+12,2.491864e+12,2.384023e+12,2.429680e+12,2.466346e+12,2.498729e+12,2.546482e+12,2.624671e+12,2.682263e+12,2.730705e+12
107,India,IND,1.367463e+11,1.418371e+11,1.459945e+11,1.547459e+11,1.662790e+11,1.618963e+11,1.618067e+11,1.744697e+11,...,1.333146e+12,1.385018e+12,1.502465e+12,1.656617e+12,1.766589e+12,1.862981e+12,1.981953e+12,2.130703e+12,2.301374e+12,2.464933e+12
27,Brazil,BRA,2.473107e+11,2.727241e+11,2.869495e+11,2.894594e+11,2.995488e+11,3.086954e+11,3.215074e+11,3.373104e+11,...,1.957112e+12,2.056814e+12,2.054225e+12,2.208872e+12,2.296662e+12,2.340784e+12,2.411121e+12,2.423272e+12,2.331933e+12,2.248106e+12
113,Italy,ITA,5.455555e+11,5.903305e+11,6.269526e+11,6.621229e+11,6.806471e+11,7.028908e+11,7.449574e+11,7.984350e+11,...,2.234494e+12,2.211023e+12,2.089813e+12,2.125058e+12,2.137312e+12,2.077061e+12,2.041166e+12,2.043486e+12,2.059491e+12,2.077613e+12
33,Canada,CAN,3.163484e+11,3.263554e+11,3.495814e+11,3.676959e+11,3.923296e+11,4.183667e+11,4.459160e+11,4.589158e+11,...,1.596798e+12,1.612771e+12,1.565201e+12,1.613464e+12,1.664147e+12,1.693194e+12,1.735101e+12,1.779611e+12,1.796369e+12,1.822736e+12


In [377]:
col_years = [str(yr) for yr in range(1960, 2017)]
# defined as _years elsewhere (I think below..)
# better to keep same definition but different use cases separate


In [378]:
top_15_gdp = get_max_countries(df_gdp, 15, '2016')
top_15_pop = get_max_countries(df_pop, 15, '2016')
top_15_pop_old = get_max_countries(df_pop, 10, '1980')
top_15_gdp_old = get_max_countries(df_gdp, 10, '1980')
top_15_gdp_all_time = df_gdp.nlargest(15, col_years) # top 5 gdp of any years
top_15_gdp.head(3)
top_30_pop_all_time = df_pop.nlargest(15, col_years)
top_20_gdp_all_time = df_gdp.nlargest(20, col_years)
top_30_gdp_all_time = df_gdp.nlargest(30, col_years)
top_50_gdp_now = get_max_countries(df_pop, 50, '2016')

top_20_pop_now = get_max_countries(df_pop, 20, '2016')
top_50_pop_now = get_max_countries(df_pop, 50, '2016')

In [379]:
#gdp_missing_countries

In [380]:
countries_with_nan_set = set([name for (name, num_missing) in gdp_missing_countries])

# not working
#is_country_in_nan_set = lambda _df: [print(row['Country Name']) for row in _df.iterrows() if row['Country Name']  
#                                     in countries_with_nan_set]

def is_country_in_nan_set(_df):
    _countries_in_nan_set = []
    for _i, country_row in _df.iterrows():
        _name = country_row['Country Name']
        if country_row['Country Name'] in countries_with_nan_set:
            _countries_in_nan_set.append(_name)
    return _countries_in_nan_set


#print(is_country_in_nan_set(top_15_gdp_all_time)) # none
#print(is_country_in_nan_set(top_15_gdp)) # Germany, Russian Federation
#print(is_country_in_nan_set(top_15_pop)) # Russian Federation, Ethiopia, Vietnma
#print(is_country_in_nan_set(top_20_gdp_all_time)) # Venezuela, RB
#print(is_country_in_nan_set(top_20_pop_now)) # 'Russian Federation', 'Ethiopia', 'Vietnam', 'Germany', 'Iran, Islamic Rep.'

print(is_country_in_nan_set(top_50_pop_now)) # Russian Federation, Germany, Ukraine


#get_missing_num_years_for_country(df_gdp, 'Ethiopia') #36



['Russian Federation', 'Ethiopia', 'Vietnam', 'Germany', 'Iran, Islamic Rep.', 'Tanzania', 'Ukraine', 'Uganda', 'Poland', 'Iraq', 'Morocco', 'Afghanistan', 'Saudi Arabia', 'Uzbekistan', 'Venezuela, RB', 'Mozambique', 'Angola', 'Yemen, Rep.']


In [383]:
# suddenly breaking with 1l0 keyError.
# get_missing_num_years_for_country(df_gdp, 'USA') #0
# get_missing_num_years_for_country(df_gdp, 'Russian Federation') # 28 data from 1989
# get_missing_num_years_for_country(df_gdp, 'Vietnam') # 33
# get_missing_num_years_for_country(df_gdp, 'Iran') # 33
# df_gdp[df_gdp['Country Name']== 'Ukraine']  # no data until 1987

KeyError: 1.0

In [ ]:
df_gdp[df_gdp['Country Name'] == 'Germany'].shape #(59 cols)
df_gdp[df_gdp['Country Name'] == 'Germany']['1980']
